# Loading data to Supabase Posgres Database

We have decided to try and host the database using Supabase as the backend. Hosting this on a Supabase's cloud, we could more easily scale and build applications that can access that database. We will eventually try this with the isochrone project.

In [2]:
import pandas as pd
import os
from supabase import create_client, Client

## Supabase API

We have created a database on Supabase off-code. This was created using my(nabilersyad) account under that organization. The database mirrors the SQLite database.
Details on the created database below.

SUPABASE_URL = 'saved locally in the shell. Get the details from supabase dashboard'  
SUPABASE_KEY = 'saved in the shell. Get the details from supabase dashboard'

database: transit-stations-database  
tables: 1. stations  
        2. entrances  
        3. station_entrances  

In [3]:
#declaring the supabase client we will working with
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

In [3]:
#signin a user
#sign_in_response = supabase.auth.sign_in_with_password({"email": "nabile95@hotmail.com", "password": ""})


In [4]:

#user = supabase.auth.get_user()


## Stations table

Combining the various CSVs of different cities into one table and we will use this table to update data in the supabase database. Assume all data in this combined table to be the most updated

In [8]:

# Define the directory of cleansed data to transfer to sql
cleansed_data_directory = 'data_cleansed'
cleansed_kl_file = 'klang_valley_stations_cleansed.csv'
cleansed_montreal_file = 'montreal_stations_cleansed.csv'
cleansed_singapore_file = 'singapore_stations_cleansed.csv'
cleansed_combined_file = 'combined_stations_cleansed.csv'

# read cleaned dataframes
kl_data = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_kl_file))
montreal_data = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_montreal_file))
singapore_data = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_singapore_file))


# Combine all the dataframes
stations_data_local = pd.concat([kl_data, montreal_data, singapore_data], axis=0, ignore_index=True)

stations_data_local.index.name = 'station_id'
stations_data_local.to_csv(os.path.join(cleansed_data_directory, cleansed_combined_file), index=True)


# Print out the combined dataframe
stations_data_local


,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened
station_id,,,,,,,,,,,,,,
0,KL Sentral,KA01,Keretapi Tanah Melayu,3.134603,101.686567,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
1,Kuala Lumpur,KA02,Keretapi Tanah Melayu,3.139513,101.693789,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
2,Bank Negara,KA03,Keretapi Tanah Melayu,3.154542,101.693010,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
3,Putra,KA04,Keretapi Tanah Melayu,3.165005,101.691234,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
4,Mid Valley,KB01,Keretapi Tanah Melayu,3.118528,101.678985,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,Outram Park MRT Station,TE17,NaN,1.280400,103.840100,TEL,Thomson-East Coast Line,NaN,Brown,#734538,Singapore,NaN,NaN,NaN
532,Maxwell MRT Station,TE18,NaN,1.280600,103.844000,TEL,Thomson-East Coast Line,NaN,Brown,#734538,Singapore,NaN,NaN,NaN
533,Shenton Way MRT Station,TE19,NaN,1.277540,103.850770,TEL,Thomson-East Coast Line,NaN,Brown,#734538,Singapore,NaN,NaN,NaN


We will now retrieve the equivalent table in supabase. This will be the table we need to update

In [9]:
response = supabase.table('stations').select("*").execute()
data,_ = response
stations_data_supa = pd.DataFrame(data[1])
stations_data_supa.set_index('station_id',inplace=True)
stations_data_supa

,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened
station_id,,,,,,,,,,,,,,
1,Kuala Lumpur,KA02,Keretapi Tanah Melayu,3.139513,101.693789,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
2,Bank Negara,KA03,Keretapi Tanah Melayu,3.154542,101.693010,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
3,Putra,KA04,Keretapi Tanah Melayu,3.165005,101.691234,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
4,Mid Valley,KB01,Keretapi Tanah Melayu,3.118528,101.678985,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
5,Seputeh,KB02,Keretapi Tanah Melayu,3.113697,101.681299,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,Maxwell MRT Station,TE18,None,1.280600,103.844000,TEL,Thomson-East Coast Line,None,Brown,#734538,Singapore,None,None,None
533,Shenton Way MRT Station,TE19,None,1.277540,103.850770,TEL,Thomson-East Coast Line,None,Brown,#734538,Singapore,None,None,None
534,Marina Bay MRT Station,TE20,None,1.275290,103.854810,TEL,Thomson-East Coast Line,None,Brown,#734538,Singapore,None,None,None


### Merging local and supabase tables (Stations)

We will merge the two databases on their index in a union merge while creating  a new column to indicate for each row, which table has it

There will be 4 cases

Case 1: Row is in both local and supa table and is identical

Case 2: Row is in both tables but has some differences in one or more columns

Case 3: rows in local but not in supa - only_local

Case 4: rows not in local but in supa - only_supa


In [10]:
# Perform an outer join on the dataframes
merged_stations_data = pd.merge(stations_data_local, stations_data_supa, how='outer',left_index=True,right_index=True,indicator=True)

# Case 1 and 2: rows exist in both dataframes
both_stations_data = merged_stations_data[merged_stations_data['_merge'] == 'both']
both_stations_data = both_stations_data.drop(columns=['_merge'])

merged_stations_data['_merge'].value_counts()



_merge
both          536
left_only       0
right_only      0
Name: count, dtype: int64

#### Case 1 handling
We will identify the rows that are in both tables and identical from the both_data table using the 'both' indicator.

This is the extend of work that needs to be done. The supabase table does not need to be updated

In [12]:
#Case 1 
identical_stations_rows_mask = [stations_data_local.loc[i, stations_data_local.columns].equals(stations_data_supa.loc[i, stations_data_supa.columns])
                        for i in both_stations_data.index]
identical_stations_rows = both_stations_data[identical_stations_rows_mask]
identical_stations_rows = stations_data_local.loc[identical_stations_rows.index]
identical_stations_rows

,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened
station_id,,,,,,,,,,,,,,
0,KL Sentral,KA01,Keretapi Tanah Melayu,3.134603,101.686567,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
1,Kuala Lumpur,KA02,Keretapi Tanah Melayu,3.139513,101.693789,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
2,Bank Negara,KA03,Keretapi Tanah Melayu,3.154542,101.693010,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
3,Putra,KA04,Keretapi Tanah Melayu,3.165005,101.691234,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
4,Mid Valley,KB01,Keretapi Tanah Melayu,3.118528,101.678985,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,Outram Park MRT Station,TE17,NaN,1.280400,103.840100,TEL,Thomson-East Coast Line,NaN,Brown,#734538,Singapore,NaN,NaN,NaN
532,Maxwell MRT Station,TE18,NaN,1.280600,103.844000,TEL,Thomson-East Coast Line,NaN,Brown,#734538,Singapore,NaN,NaN,NaN
533,Shenton Way MRT Station,TE19,NaN,1.277540,103.850770,TEL,Thomson-East Coast Line,NaN,Brown,#734538,Singapore,NaN,NaN,NaN


#### Case 2 handling
Using the mask from handling case 1, we can get the rows with different values by getting the inverse of the identical_rows_mask to get right index. Then using this index to select the relevant rows from both_data

We will then use the rows from different_rows to update the supabase table using the update() function

In [13]:
different_stations_rows_mask= [not b for b in identical_stations_rows_mask]
different_stations_rows = both_stations_data[different_stations_rows_mask]

# Replace all NaN values in the DataFrame with None
different_stations_rows = different_stations_rows.where(pd.notna(different_stations_rows), None)
different_stations_rows = stations_data_local.loc[different_stations_rows.index]

different_stations_rows

,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened
station_id,,,,,,,,,,,,,,


In [14]:
#the following is to convert index in pandas to be a column, usable for posgres
different_stations_rows = different_stations_rows.reset_index(drop=False)

# For the different rows, you want to update data_supabase with data from data_local
for index, row in different_stations_rows.iterrows():
    # Use Supabase update method
    # Note: replace 'id' and 'your_table' with your actual id column name and table name
    data, error = supabase.table('stations').update(row.to_dict()).eq('station_id', row['station_id']).execute()


#### Case 3 handling
Using the left_only indicator, we can get the rows that are in local that needs to be inserted in the supabase table

We will then use the rows from only_local to update the supabase table using the insert() function

In [15]:

# Case 3: rows in data_local but not in data_supabase

only_local_stations = merged_stations_data[merged_stations_data['_merge'] == 'left_only']
only_local_stations = only_local_stations.drop(columns=['_merge'])

only_local_stations = stations_data_local.loc[only_local_stations.index]
only_local_stations = only_local_stations.where(pd.notna(only_local_stations), None)
only_local_stations = only_local_stations.reset_index(drop=False)

only_local_stations

,station_id,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened


In [16]:
# For these rows, you want to insert into data_supabase
data, error = supabase.table('stations').insert(only_local_stations.to_dict('records')).execute()

#### Case 4 handling
Using the right_only indicator, we can get the rows that are in supabase that needs to be removed from the supabase table

We will then use the rows from only_local to update the supabase table using the delete() function

In [17]:
# Case 4: rows not in data_local but in data_supabase
only_supabase_stations = merged_stations_data[merged_stations_data['_merge'] == 'right_only']
only_supabase_stations = only_supabase_stations.drop(columns=['_merge'])

only_supabase_stations = stations_data_local.loc[only_supabase_stations.index]
only_supabase_stations = only_supabase_stations.where(pd.notna(only_supabase_stations), None)
only_supabase_stations = only_supabase_stations.reset_index(drop=False)
only_supabase_stations

,station_id,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened


In [18]:
# For these rows, you want to delete from data_supabase
for index, row in only_supabase_stations.iterrows():
    # Use Supabase delete method
    data, error = supabase.table('stations').delete().eq('station_id', row['station_id']).execute()

## Entrances tables

There are two entrances tables that we need to create. We'll call them entrances and station_entrances.
Again assume all local data to be the most updated

We will start with entrances table

In [5]:
# Retrieve entrances table in local csv file
cleansed_entrances = 'klang_valley_entrances_cleansed.csv'

# read cleaned entrances dataframes
entrances_data_local = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_entrances))

#ensure NAs are in a posgres readable format
entrances_data_local = entrances_data_local.where(pd.notna(entrances_data_local), None)

#ensure index to be entrance_id because it is easier to work with
entrances_data_local.set_index('entrance_id',inplace=True)
entrances_data_local = entrances_data_local.sort_values(by='entrance_id')

entrances_data_local

,longitude,latitude,entrance_destination,entrance_name
entrance_id,,,,
1544031348,101.711374,3.145929,None,B
1631412559,101.604952,3.113222,None,None
2278515570,101.644077,3.050650,None,None
2686635178,101.699182,3.138565,None,C
3308608988,101.712717,3.158762,None,None
...,...,...,...,...
11052165924,101.694160,3.161349,None,None
11061429683,101.671755,3.214623,None,None
11061429685,101.672075,3.214698,None,None


In [6]:
# Retrieve entrances table from supabase

response = supabase.table('entrances').select("*").execute()
data,_ = response
entrances_data_supa = pd.DataFrame(data[1])

#ensure index to be entrance_id because it is easier to work with
entrances_data_supa.set_index('entrance_id',inplace=True)
entrances_data_supa = entrances_data_supa.sort_values(by='entrance_id')

entrances_data_supa

,longitude,latitude,entrance_destination,entrance_name
entrance_id,,,,
1544031348,101.711374,3.145929,None,B
1631412559,101.604952,3.113222,None,None
2278515570,101.644077,3.050650,None,None
2686635178,101.699182,3.138565,None,C
3308608988,101.712717,3.158762,None,None
...,...,...,...,...
11052165924,101.694160,3.161349,None,None
11061429683,101.671755,3.214623,None,None
11061429685,101.672075,3.214698,None,None


### Merging local and supabase tables

We will merge the two databases on their index in a union merge while creating  a new column to indicate for each row, which table has it

There will be 4 cases

Case 1: Row is in both local and supa table and is identical

Case 2: Row is in both tables but has some differences in one or more columns

Case 3: rows in local but not in supa - only_local

Case 4: rows not in local but in supa - only_supa

In [88]:
# Perform an outer join on the dataframes
merged_data = pd.merge(entrances_data_local,entrances_data_supa,how='outer',left_index=True,right_index=True,indicator=True)

# Case 1 and 2: rows exist in both dataframes
both_data = merged_data[merged_data['_merge'] == 'both']
both_data = both_data.drop(columns=['_merge'])

merged_data['_merge'].value_counts()


#### Case 1 handling
We will identify the rows that are in both tables and identical from the both_data table using the 'both' indicator.

This is the extend of work that needs to be done. The supabase table does not need to be updated

In [110]:
#Case 1 
identical_rows_mask = [entrances_data_local.loc[i, entrances_data_local.columns].equals(entrances_data_supa.loc[i, entrances_data_supa.columns])
                        for i in both_data.index]
identical_rows = both_data[identical_rows_mask]
identical_rows = entrances_data_local.loc[identical_rows.index]
identical_rows

,longitude,latitude,entrance_destination,entrance_name
entrance_id,,,,
1544031348,101.711374,3.145929,None,B
1631412559,101.604952,3.113222,None,None
2278515570,101.644077,3.050650,None,None
2686635178,101.699182,3.138565,None,C
3308608988,101.712717,3.158762,None,None
...,...,...,...,...
10830050822,101.687762,3.132827,None,C
10839997852,101.657207,2.949712,Off Persiaran APEC,A
10864116957,101.731475,3.165190,None,None


#### Case 2 handling
Using the mask from handling case 1, we can get the rows with different values by getting the inverse of the identical_rows_mask to get right index. Then using this index to select the relevant rows from both_data

We will then use the rows from different_rows to update the supabase table using the update() function

In [114]:
different_rows_mask= [not b for b in identical_rows_mask]
different_rows = both_data[different_rows_mask]

# Replace all NaN values in the DataFrame with None
different_rows = different_rows.where(pd.notna(different_rows), None)
different_rows = entrances_data_local.loc[different_rows.index]

different_rows

,longitude,latitude,entrance_destination,entrance_name
entrance_id,,,,
4952299496,101.729057,3.111715,Velodrom KL;Jalan Cheras,B
5261535560,101.607848,3.155939,Jalan PJU 7/1,C
5261535561,101.608202,3.155933,Jalan PJU 7/1,B
5261535570,101.609118,3.154899,Persiaran Surian,A
5261535576,101.609131,3.155588,Surian Tower;The Curve,D
5386930796,101.695428,3.172515,Hentian Bas Titiwangsa,None
5392028001,101.719112,3.159893,None,None
5469085305,101.774599,3.048335,Bandar Tun Hussein Onn,A
5469085306,101.774288,3.048340,Park & Ride,B


In [116]:
#the following is to convert index in pandas to be a column, usable for posgres
different_rows = different_rows.reset_index(drop=False)

# For the different rows, you want to update data_supabase with data from data_local
for index, row in different_rows.iterrows():
    # Use Supabase update method
    # Note: replace 'id' and 'your_table' with your actual id column name and table name
    data, error = supabase.table('entrances').update(row.to_dict()).eq('entrance_id', row['entrance_id']).execute()


#### Case 3 handling
Using the left_only indicator, we can get the rows that are in local that needs to be inserted in the supabase table

We will then use the rows from only_local to update the supabase table using the insert() function

In [122]:

# Case 3: rows in data_local but not in data_supabase

only_local = merged_data[merged_data['_merge'] == 'left_only']
only_local = only_local.drop(columns=['_merge'])

only_local = entrances_data_local.loc[only_local.index]
only_local = only_local.where(pd.notna(only_local), None)
only_local = only_local.reset_index(drop=False)

only_local

,entrance_id,longitude,latitude,entrance_destination,entrance_name
0,5737775567,101.752959,3.138617,Taman Cahaya;Taman Nirwana,None
1,5759847421,101.663256,3.144283,None,KL2333
2,7629425122,101.756469,3.140678,None,None
3,9913027320,101.695604,3.185932,None,None
4,9942835610,100.366719,5.393692,None,None
5,10246792131,101.740727,3.091086,None,C
6,10941247782,101.752945,3.138276,Taman Cahaya;Pandan Indah,None
7,11032999050,101.706130,3.102146,Salak Selatan;Pekan Salak Selatan;Desa Petaling,None
8,11032999053,101.706463,3.102026,Bandar Sri Permaisuri,None
9,11033916949,101.593926,3.149417,Tropicana Gardens Mall,None


In [123]:
# For these rows, you want to insert into data_supabase
data, error = supabase.table('entrances').insert(only_local.to_dict('records')).execute()

#### Case 4 handling
Using the right_only indicator, we can get the rows that are in supabase that needs to be removed from the supabase table

We will then use the rows from only_local to update the supabase table using the delete() function

In [126]:
# Case 4: rows not in data_local but in data_supabase
only_supabase = merged_data[merged_data['_merge'] == 'right_only']
only_supabase = only_supabase.drop(columns=['_merge'])

only_supabase = entrances_data_supa.loc[only_supabase.index]
only_supabase = only_supabase.where(pd.notna(only_supabase), None)
only_supabase = only_supabase.reset_index(drop=False)
only_supabase

,entrance_id,longitude,latitude,entrance_destination,entrance_name
0,5469085434,101.740791,3.091162,None,C


In [127]:
# For these rows, you want to delete from data_supabase
for index, row in only_supabase.iterrows():
    # Use Supabase delete method
    data, error = supabase.table('entrances').delete().eq('entrance_id', row['entrance_id']).execute()

## Station Entrances tables

This table tracks the relationship between station and entrances

In [20]:
# Retrieve station entrances table in local csv file
cleansed_station_entrances= 'klang_valley_stations_entrances_relation_cleansed.csv'

# read cleaned statuion entrances dataframes
station_entrances_data_local = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_station_entrances))

#ensure NAs are in a posgres readable format
station_entrances_data_local = station_entrances_data_local.where(pd.notna(station_entrances_data_local), None)

#ensure index to be entrance_id because it is easier to work with
station_entrances_data_local.set_index('relationship_id',inplace=True)
station_entrances_data_local = station_entrances_data_local.sort_values(by='entrance_id')

station_entrances_data_local


,entrance_id,station_name,station_code
relationship_id,,,
137,1544031348,Bukit Bintang (Monorail),MR6
94,1631412559,Kelana Jaya,KJ24
85,2278515570,Kinrara BK 5,SP22
173,2686635178,Maharajalela,MR3
23,3308608988,KLCC,KJ10
...,...,...,...
149,11039725697,Masjid Jamek (AG SP),AG7
177,11061429683,Taman Wahyu,KC04
176,11061429685,Taman Wahyu,KC04


In [21]:
# Retrieve entrances table from supabase

response = supabase.table('station_entrances').select("*").execute()
data,_ = response
station_entrances_data_supa = pd.DataFrame(data[1])

#ensure index to be entrance_id because it is easier to work with
station_entrances_data_supa.set_index('relationship_id',inplace=True)
station_entrances_data_supa = station_entrances_data_supa.sort_values(by='relationship_id')

station_entrances_data_supa

,entrance_id,station_name,station_code
relationship_id,,,
0,10796851698,Pudu,AG10
1,10796851698,Pudu,SP10
2,5485710279,Kampung Baru,KJ11
3,5485710278,Kampung Baru,KJ11
4,9740843587,Masjid Jamek (KJ),KJ13
...,...,...,...
239,10839997852,Cyberjaya Utara,PY39
240,10658294223,Cyberjaya City Centre,PY30
241,10722980582,Putrajaya Sentral (MRT),PY41


### Merging local and supabase tables

We will merge the two databases on their index in a union merge while creating  a new column to indicate for each row, which table has it

There will be 4 cases

Case 1: Row is in both local and supa table and is identical

Case 2: Row is in both tables but has some differences in one or more columns

Case 3: rows in local but not in supa - only_local

Case 4: rows not in local but in supa - only_supa

In [22]:
# Perform an outer join on the dataframes
merged_station_entrances_data = pd.merge(station_entrances_data_local,station_entrances_data_supa,how='outer',left_index=True,right_index=True,indicator=True)

# Case 1 and 2: rows exist in both dataframes
both_station_entrances_data = merged_station_entrances_data[merged_station_entrances_data['_merge'] == 'both']
both_station_entrances_data = both_station_entrances_data.drop(columns=['_merge'])

merged_station_entrances_data['_merge'].value_counts()


_merge
both          244
left_only       2
right_only      0
Name: count, dtype: int64

#### Case 1 handling
We will identify the rows that are in both tables and identical from the both_data table using the 'both' indicator.

This is the extend of work that needs to be done. The supabase table does not need to be updated

In [24]:
#Case 1 
identical_station_entrances_rows_mask = [station_entrances_data_local.loc[i, station_entrances_data_local.columns].equals(station_entrances_data_supa.loc[i, station_entrances_data_supa.columns])
                        for i in both_station_entrances_data.index]
identical_station_entrances_rows = both_station_entrances_data[identical_station_entrances_rows_mask]
identical_station_entrances_rows = station_entrances_data_local.loc[identical_station_entrances_rows.index]
identical_station_entrances_rows

,entrance_id,station_name,station_code
relationship_id,,,
0,10796851698,Pudu,AG10
1,10796851698,Pudu,SP10
2,5485710279,Kampung Baru,KJ11
3,5485710278,Kampung Baru,KJ11
4,9740843587,Masjid Jamek (KJ),KJ13
5,9983121350,Masjid Jamek (KJ),KJ13
6,11039725697,Masjid Jamek (KJ),KJ13
7,5044809567,Pasar Seni (KG),KG16
8,5044809566,Pasar Seni (KG),KG16


#### Case 2 handling
Using the mask from handling case 1, we can get the rows with different values by getting the inverse of the identical_rows_mask to get right index. Then using this index to select the relevant rows from both_data

We will then use the rows from different_rows to update the supabase table using the update() function

In [26]:
different_station_entrances_rows_mask= [not b for b in identical_station_entrances_rows_mask]
different_station_entrances_rows = both_station_entrances_data[different_station_entrances_rows_mask]

# Replace all NaN values in the DataFrame with None
different_station_entrances_rows = different_station_entrances_rows.where(pd.notna(different_station_entrances_rows), None)
different_station_entrances_rows = station_entrances_data_local.loc[different_station_entrances_rows.index]

different_station_entrances_rows

,entrance_id,station_name,station_code
relationship_id,,,
16,9913027320,Sentul Timur,AG1
17,9913027320,Sentul Timur,SP1
18,5386930796,Titiwangsa (LRT),AG3
19,5386930796,Titiwangsa (LRT),SP3
20,10860751179,Titiwangsa (LRT),AG3
...,...,...,...
239,10266244039,Putra Permai,PY37
240,10611678882,16 Sierra,PY38
241,10839997852,Cyberjaya Utara,PY39


#### Case 3 handling
Using the left_only indicator, we can get the rows that are in local that needs to be inserted in the supabase table

We will then use the rows from only_local to update the supabase table using the insert() function

In [27]:

# Case 3: rows in data_local but not in data_supabase

only_local_station_entrances = merged_station_entrances_data[merged_station_entrances_data['_merge'] == 'left_only']
only_local_station_entrances = only_local_station_entrances.drop(columns=['_merge'])

only_local_station_entrances = station_entrances_data_local.loc[only_local_station_entrances.index]
only_local_station_entrances = only_local_station_entrances.where(pd.notna(only_local_station_entrances), None)
only_local_station_entrances = only_local_station_entrances.reset_index(drop=False)

only_local_station_entrances

,relationship_id,entrance_id,station_name,station_code
0,244,5044809585,Tun Razak Exchange (PY),PY23
1,245,5044809586,Tun Razak Exchange (PY),PY23


In [29]:
# For these rows, you want to insert into data_supabase
data, error = supabase.table('station_entrances').insert(only_local_station_entrances.to_dict('records')).execute()

#### Case 4 handling
Using the right_only indicator, we can get the rows that are in supabase that needs to be removed from the supabase table

We will then use the rows from only_local to update the supabase table using the delete() function

In [30]:
# Case 4: rows not in data_local but in data_supabase
only_supabase_station_entrances = merged_station_entrances_data[merged_station_entrances_data['_merge'] == 'right_only']
only_supabase_station_entrances = only_supabase_station_entrances.drop(columns=['_merge'])

only_supabase_station_entrances = station_entrances_data_local.loc[only_supabase_station_entrances.index]
only_supabase_station_entrances = only_supabase_station_entrances.where(pd.notna(only_supabase_station_entrances), None)
only_supabase_station_entrances = only_supabase_station_entrances.reset_index(drop=False)
only_supabase_station_entrances

,relationship_id,entrance_id,station_name,station_code


In [31]:
# For these rows, you want to delete from data_supabase
for index, row in only_supabase_station_entrances.iterrows():
    # Use Supabase delete method
    data, error = supabase.table('station_entrances').delete().eq('relationship_id', row['relationship_id']).execute()